In [1]:
# import Libraries

import openai
import langchain
import pinecone
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()


False

In [7]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents
doc=read_doc('documents/')
len(doc)

58

Loaded budget_speech.pdf from documents folder

In [8]:
## Divide the docs into chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [9]:
documents=chunk_data(docs=doc)
len(documents)

58

In [27]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = "hkunlp/instructor-large"

# Get the Hugging Face Hub API token
hf_api_token = os.getenv('HF_TOKEN')

embeddings1 = HuggingFaceEmbeddings(
    model_name=model,
    huggingfacehub_api_token=hf_api_token
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [28]:
vectors=embeddings1.embed_query("How are you?")
len(vectors)

768

Now Setting up our Pinecone DB

In [ ]:
from pinecone import Pinecone
from dotenv import load_dotenv

# Get the Pinecone API key from the environment variables
pinecone_api_key = os.getenv('PINECONE_API_KEY')

## Vector Search DB In Pinecone
pinecone = Pinecone(
    api_key=pinecone_api_key
)
index_name="langchainvector"

In [46]:
import os
from langchain_pinecone import PineconeVectorStore

# Get the Pinecone API key from the environment variables
pinecone_api_key = os.getenv('PINECONE_API_KEY')

# Now you can create the PineconeVectorStore without passing the API key directly
index = PineconeVectorStore.from_documents(
    doc,
    embeddings1,
    index_name=index_name,
    pinecone_api_key=pinecone_api_key
)

In [47]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [50]:
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq

# Get the Groq API key from the environment variables
groq_api_key = os.getenv('GROQ_API_KEY')

llm=ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0.5,
    api_key=groq_api_key # Pass the API key retrieved from .env
)
chain=load_qa_chain(llm,chain_type="stuff")

<ipython-input-50-cc4b85e50457>:8: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain=load_qa_chain(llm,chain_type="stuff")


In [51]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

Final output from our RAG Pipeline

In [52]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

[Document(id='5032304d-34d3-48fa-8f17-15bcbbe8790f', metadata={'page': 10.0, 'source': 'documents/budget_speech.pdf'}, page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  \nFisheries \n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana \nwith targeted investment of ` 6,000 crore to further enable activities of \nfishermen, fish vendors, and micro & small enterprises, improve value chain \nefficiencies, and expand the market. \nCooperation \n25. For farmers, especially small and marginal farmers, and other \nmarginalise

<ipython-input-51-d2374476e434>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain.run(input_documents=doc_search,question=query)


According to the text, the agriculture credit target will be increased to ₹ 20 lakh crore with a focus on animal husbandry, dairy, and fisheries.


Final Output of Retreived Data Shown in Last Line of Output